<a href="https://colab.research.google.com/github/Davottakvota/VKRGAN/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки

In [ ]:
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython.display import HTML
from IPython import display
import tensorflow_datasets as tfds
import cv2
!pip install vnoise
import vnoise

tf.__version__

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# # Create a folder in the root directory
# !mkdir -p "/content/drive/My Drive/My Folder"

# # Write a sample file to Google Drive
# with open('/content/drive/My Drive/My Folder/sample.txt', 'w') as f:
#   f.write('Hello, World!')

Создание папок

In [ ]:
%%bash
dirs=(
    "/content/drive/My Drive/checkweights"
    "/content/drive/My Drive/checkweights2"
    "/content/drive/My Drive/videos"
    "/content/drive/My Drive/videos/5epochs"
    "/content/drive/My Drive/videos/20epochs"
    "/content/drive/My Drive/videos/60epochs"
    "/content/drive/My Drive/videos/SAMPLESFORMETRIC"
)

for dir in "${dirs[@]}"; do
    if [ ! -d "$dir" ]; then
        mkdir -p "$dir"
        echo "✅ Создана папка: $dir"
    else
        echo "ℹ️ Папка уже существует: $dir"
    fi
done

In [ ]:
#!rm –f "/content/drive/My Drive/videos/TEST.mp4"

# GAN

## Генератор

In [ ]:
FPS=4
duration=5
width=64
height=64
noisedim=100

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(int(FPS*duration/4)*int(width/8)*int(height/8)*512, use_bias=False, input_shape=(noisedim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.Reshape((int(FPS*duration/4), int(width/8), int(height/8), 512)))
    assert model.output_shape == (None, int(FPS*duration/4), int(width/8), int(height/8), 512)  # Note: None is the batch size

    model.add(layers.Conv3DTranspose(256, (5, 5, 5), strides=(1, 1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, int(FPS*duration/4), int(width/8), int(height/8), 256)
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.Conv3DTranspose(128, (5, 5, 5), strides=(1, 2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, int(FPS*duration/4), int(width/4), int(height/4), 128)
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.Conv3DTranspose(64, (5, 5, 5), strides=(2, 2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, int(FPS*duration/2), int(width/2), int(height/2), 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.Conv3DTranspose(1, (5, 5, 5), strides=(2, 2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, int(FPS*duration), int(width), int(height), 1)

    return model

#### Noise

In [ ]:
def gennoise(n):
  # Шум Перлина
  # numPoints = 100
  # time = np.linspace(0, 10, num=numPoints)
  # noise = vnoise.Noise()
  # noises = []
  # for i in range(n):
  #   seed = np.random.randint(0, 1000)
  #   noise.seed(seed)
  #   noises.append(noise.noise1(time, octaves=20))
  # return np.array(noises)

  # Нормальный шум
  return tf.random.normal([n, noisedim])

#### Generation test

In [ ]:
generator = make_generator_model()

noise = gennoise(1)
generated_image = generator(noise, training=False)

generated_image.shape

In [ ]:
fig, ax = plt.subplots()
def animate(i):
    ax.clear()
    return ax.imshow(generated_image[0, i, :, :, 0], cmap='gray')
ani = animation.FuncAnimation(fig=fig,
                              func=animate,
                              frames=generated_image[0, :, 0, 0, 0].shape[0],
                              interval=1000/FPS,
                              blit=False)
ani.save('/content/drive/My Drive/videos/TEST.mp4', animation.FFMpegWriter(fps=2) )
HTML(ani.to_jshtml())

## Дискриминатор

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(int(FPS*duration), width, height, 1)))

    model.add(layers.Conv3D(64, (5, 5, 5), strides=(2, 2, 2), padding='same'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Conv3D(128, (3, 3, 3), strides=(2, 2, 2), padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv3D(256, (3, 3, 3), padding='same'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.GlobalAveragePooling3D())

    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

## Кросс-энтропия

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

**Изменена скорость обучения для дискриминатора + бэта**

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=4e-4, beta_1=0.5)

## Чекпоинт

In [ ]:
checkpoint_dir = "/content/drive/My Drive/checkweights"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
#READ
checkpoint.read(checkpoint_prefix)

#### Only Generator chekpoint save

In [ ]:
#!mkdir -p "/content/drive/My Drive/checkweights/generatorweights"

In [ ]:
#generator.save_weights('/content/drive/My Drive/checkweights/generatorweights/model.weights.h5')

# moving mnist

In [ ]:
BATCH_SIZE = 100
ds = tfds.load('moving_mnist', split='test', batch_size=BATCH_SIZE)

In [ ]:
videos = tfds.as_numpy(ds)

In [ ]:
for iv in videos:
  iv = iv['image_sequence']
  fig, ax = plt.subplots()
  def animate(i):
      ax.clear()
      return ax.imshow(iv[0, i, :, :, 0], cmap='gray')
  ani = animation.FuncAnimation(fig=fig,
                                func=animate,
                                frames=iv[0, :, 0, 0, 0].shape[0],
                                interval=1000/FPS,
                                blit=False)
  #ani.save("/content/drive/My Drive/videos/LEARNSAMPLE.mp4", animation.FFMpegWriter(fps=2) )
  break
HTML(ani.to_jshtml())

In [ ]:
# for iv in videos:
#   iv = iv['image_sequence']

#   for jj in range(len(iv)):
#     vid = np.uint8(iv[jj])

#     frame_array = list(vid)
#     # Set up the video writer
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter('/content/drive/My Drive/videos/GTVIDEOS/GT'+str(jj)+'.mp4', fourcc, 2.0, (64, 64), isColor=False)
#     # Write frames to the video file
#     for frame in frame_array:
#         out.write(frame)

#     out.release()
#   break

# Train

In [ ]:
EPOCHS = 60
BATCH_SIZE

In [ ]:
@tf.function
def train_step(images):
    noise = gennoise(BATCH_SIZE)
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
import cv2
import numpy as np

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    #Эпоха
    j = 0
    for image_batch in dataset:
        print("\r", j, end='')
        j+=1
        d = image_batch['image_sequence']
        d = (d-127.5)/127.5
        train_step(d)

        # if i>10:
        #   break
    print()

    #Чекпоинт
    !echo '' > "/content/drive/My Drive/checkweights/ckpt.data-00000-of-00001" && rm "/content/drive/My Drive/checkweights/ckpt.data-00000-of-00001"
    checkpoint.write(file_prefix = checkpoint_prefix)
    !mkdir -p "/content/drive/My Drive/checkweights2" && cp "/content/drive/My Drive/checkweights/ckpt.data-00000-of-00001" "/content/drive/My Drive/checkweights2/"

    #50 видео
    if (epoch==4 or epoch==19):

      noise = gennoise(1)
      generated_image = generator(noise, training=False)
      fig, ax = plt.subplots()
      def animate(i):
          ax.clear()
          return ax.imshow(generated_image[0, i, :, :, 0], cmap='gray')
      ani = animation.FuncAnimation(fig=fig,
                                    func=animate,
                                    frames=generated_image[0, :, 0, 0, 0].shape[0],
                                    interval=1000/FPS,
                                    blit=False)
      ani.save("/content/drive/My Drive/videos/"+str(epoch+1)+"GAN3DPlot.mp4", animation.FFMpegWriter(fps=2) )
      HTML(ani.to_jshtml())

      number_of_examples = 50
      noise = gennoise(number_of_examples)
      generated_image = generator(noise, training=False)
      gvids = generated_image.numpy()
      for i in range(len(gvids)):
        vid = 127.5*gvids[i]+127.5
        vid = np.uint8(vid)

        frame_array = list(vid)
        # Set up the video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('/content/drive/My Drive/videos/'+str(epoch+1)+'epochs/output'+str(i)+'.mp4', fourcc, 4.0, (64, 64), isColor=False)
        # Write frames to the video file
        for frame in frame_array:
            out.write(frame)

        out.release()


    # #Тестовое видео
    # noise = tf.random.normal([1, noisedim])
    # generated_image = generator(noise, training=False)
    # fig, ax = plt.subplots()
    # def animate(i):
    #     ax.clear()
    #     return ax.imshow(generated_image[0, i, :, :, 0], cmap='gray')
    # ani = animation.FuncAnimation(fig=fig,
    #                               func=animate,
    #                               frames=generated_image[0, :, 0, 0, 0].shape[0],
    #                               interval=1000/FPS,
    #                               blit=False)
    # ani.save("/content/drive/My Drive/videos/GAN3DPlot.mp4", animation.FFMpegWriter(fps=2) )
    # HTML(ani.to_jshtml())


    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
train(videos, EPOCHS)

## Task

In [ ]:
noise = gennoise(1)

In [ ]:
generated_image = generator(noise, training=False)
fig, ax = plt.subplots()
def animate(i):
    ax.clear()
    return ax.imshow(generated_image[0, i, :, :, 0], cmap='gray')
ani = animation.FuncAnimation(fig=fig,
                              func=animate,
                              frames=generated_image[0, :, 0, 0, 0].shape[0],
                              interval=1000/FPS,
                              blit=False)
ani.save("/content/drive/My Drive/videos/GAN3DPlot.mp4", animation.FFMpegWriter(fps=4) )
HTML(ani.to_jshtml())

# Метрики

In [ ]:
number_of_examples = 50

In [ ]:
noise = gennoise(number_of_examples)

In [ ]:
generated_image = generator(noise, training=False)

In [ ]:
gvids = generated_image.numpy()

In [ ]:
gvids.shape

In [ ]:
import cv2
import numpy as np

In [ ]:
for i in range(len(gvids)):
  vid = 127.5*gvids[i]+127.5
  vid = np.uint8(vid)

  frame_array = list(vid)
  # Set up the video writer
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter('/content/drive/My Drive/videos/'+str(60)+'epochs/output'+str(i)+'.mp4', fourcc, 4.0, (64, 64), isColor=False)
  # Write frames to the video file
  for frame in frame_array:
      out.write(frame)

  out.release()

In [ ]:
discroutfake = discriminator(gvids).numpy()

In [ ]:
for iv in videos:
  iv = iv['image_sequence'][:50]
  break

iv.shape

In [ ]:
discroutreal = discriminator(iv).numpy()
discroutreal.shape

In [ ]:
discroutrealprob = (np.tanh(discroutreal/2)+np.ones_like(discroutreal))/2

In [ ]:
discroutfakeprob = (np.tanh(discroutfake/2)+np.ones_like(discroutfake))/2

## ACD

Average Content Distance (ACD) — это мера, которая показывает среднее расстояние между содержанием кадров в видео.  

In [ ]:
def ACD(framess):
  N = np.multiply.reduce(framess.shape[1:-1])
  res = np.mean(
              np.linalg.norm(
                np.diff(
                  np.einsum('ijkl->il', framess),
                axis=0) / N,
              axis=1)
            )
  return res

In [ ]:
sumres = 0
for i in range(number_of_examples):
  frames = generated_image[i]
  res = ACD(frames)
  sumres+=res


In [ ]:
frames.shape

In [ ]:
 np.diff(np.einsum('ijkl->il', frames), axis=0)

In [ ]:
averageACD = sumres/number_of_examples

In [ ]:
print("Для сгенерированных видео:", averageACD)

Для 10 эпох – 0.2229950550198555

60 эпох – 0.38825045645236966

In [ ]:
for imb in videos:
  h=[]
  for i in range(50):
    exvid = imb['image_sequence'][i]
    exvid = (exvid-127.5)/127.5
    h.append(ACD(exvid))
  break
print("Для тестовых видео:", np.mean(h))

## CLIP Tem-Con

CLIP Tem-Con — это метрика, которая измеряет временную согласованность путём вычисления косинусного сходства между всеми парами последовательных кадров.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

In [ ]:
def CTC(vid):
  sumFrameTC = 0
  for j in range(len(vid)-1):
    a = np.array(vid[j]).flatten()
    b = np.array(vid[j+1]).flatten()
    sumFrameTC += dot(a, b)/(norm(a)*norm(b))
  averageFrameCS = sumFrameTC/(len(frames)-1)
  return averageFrameCS

In [ ]:
sumTC = 0
for i in range(number_of_examples):
  frames = generated_image[i]
  sumFrameTC = 0
  averageFrameCS = CTC(frames)
  sumTC += averageFrameCS
averageTemCon = sumTC/number_of_examples

In [ ]:
print("Для сгенерированных видео:", averageTemCon)

Для 10 эпох – 0.1906326056926245

60 эпох – 0.3052954191273373

In [ ]:
for imb in videos:
  h=[]
  for i in range(50):
    exvid = imb['image_sequence'][i]
    exvid = (exvid-127.5)/127.5
    h.append(CTC(exvid))
  break
print("Для тестовых видео:", np.mean(h))

#MSE/RMSE

In [ ]:
realerror = (np.ones_like(discroutrealprob)-discroutrealprob)**2
fakeerror = (np.zeros_like(discroutfakeprob)-discroutfakeprob)**2

In [ ]:
print("Ошибка на реальных видео", np.sum(realerror)/50)

In [ ]:
print("Ошибка на фейковых видео", np.sum(fakeerror)/50)

In [ ]:
print("Суммарная ошибка", (np.sum(fakeerror)+np.sum(realerror))/100)

#MAE

In [ ]:
realerror = np.abs(np.ones_like(discroutrealprob)-discroutrealprob)
fakeerror = np.abs(np.zeros_like(discroutfakeprob)-discroutfakeprob)

In [ ]:
print("Ошибка на реальных видео", np.sum(realerror)/50)

In [ ]:
print("Ошибка на фейковых видео", np.sum(fakeerror)/50)

In [ ]:
print("Суммарная ошибка", (np.sum(fakeerror)+np.sum(realerror))/100)